In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from xgboost import XGBRegressor

In [2]:
df=pd.read_excel(r"../dataset/data.xlsx",sheet_name="Sheet1")
df

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350
...,...,...,...,...,...,...,...
4021,22,66,6,6.0,50,25.00,1.628000
4022,22,66,6,6.0,75,25.00,2.714000
4023,22,66,6,6.0,100,25.00,2.879000
4024,22,66,6,7.5,0,25.00,0.602680


In [3]:
df.shape

(4026, 7)

In [4]:
df.head()

,Number of Storeys(NS),Height of Structure(HS),Number of Spans(HSP),Length of Spans (LS),Opening percentage (OP),Masonry wall Stiffeness Et (MS),Period (P)
0,1,3,2,3.0,0,2.25,0.092970
1,1,3,2,3.0,100,2.25,0.159750
2,1,3,2,4.5,0,2.25,0.094770
3,1,3,2,4.5,100,2.25,0.156589
4,1,3,2,6.0,0,2.25,0.106350


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(df)

normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

print(normalized_df)

      Number of Storeys(NS)  Height of Structure(HS)  Number of Spans(HSP)  \
0                       0.0                      0.0                   0.0   
1                       0.0                      0.0                   0.0   
2                       0.0                      0.0                   0.0   
3                       0.0                      0.0                   0.0   
4                       0.0                      0.0                   0.0   
...                     ...                      ...                   ...   
4021                    1.0                      1.0                   1.0   
4022                    1.0                      1.0                   1.0   
4023                    1.0                      1.0                   1.0   
4024                    1.0                      1.0                   1.0   
4025                    1.0                      1.0                   1.0   

      Length of Spans (LS)  Opening percentage (OP)  \
0       

In [6]:
normalized_df.keys()

Index(['Number of Storeys(NS)', 'Height of Structure(HS)',
       'Number of Spans(HSP)', 'Length of Spans (LS)',
       'Opening percentage (OP)', 'Masonry wall Stiffeness Et (MS)',
       'Period (P)'],
      dtype='object')

In [7]:
X=normalized_df.drop("Period (P)" ,axis=1)

In [8]:
#标签
y=normalized_df["Period (P)"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=99)

In [10]:
import optuna
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
import optuna.integration.lightgbm as oplgb
import xgboost as xgb

In [26]:
from sklearn.model_selection import KFold,cross_validate
from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,partial
from hyperopt.early_stop import no_progress_loss


In [27]:
def bayesopt_objective(n_estimators,max_depth,subsample,learning_rate):
    model= XGBRegressor(n_estimators=int(n_estimators)
              ,max_depth=int(max_depth)
              ,subsample=int(subsample)
              ,learning_rate=learning_rate
              )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                   ,error_score='raise')
    
    return np.mean(abs(validate_loss['test_score']))

In [28]:
def param_bayes_opt(init_points,n_iter):
    opt=BayesianOptimization(bayesopt_objective
                             ,param_grid_simple
                             ,random_state=99)
 
    opt.maximize(init_points=init_points 
                 ,n_iter=n_iter 
                )
    
   
    params_best=opt.max['params'] 
    score_best=opt.max['target'] 
    
    print("\n","best params: ", params_best,
          "\n","best cvscore: ", score_best)
    
    return params_best,score_best

In [29]:
def bayes_opt_validation(params_best):
    model= XGBRegressor(n_estimators=int(params_best['n_estimators'])
            ,max_depth=int(params_best['max_depth'])
            ,subsample=int(params_best['subsample'])
            ,learning_rate=params_best['learning_rate']
           )
    cv=KFold(n_splits=10,shuffle=True,random_state=99)
    validate_loss=cross_validate(model,X_train,y_train
                                   ,cv=cv
                                   ,scoring='neg_root_mean_squared_error'
                                   ,n_jobs=-1
                                  )
    
    return np.mean(abs(validate_loss['test_score']))

In [30]:
# Start time
start_time = time.time()

param_grid_simple={'n_estimators':(800,1500)
                  ,'max_depth':(2,10)
                  ,'subsample':(0.01,1)
                   ,'learning_rate':( 0.001,0.999)
                  }

params_best,score_best=param_bayes_opt(20,100)

params_best 
score_best
validation_score=bayes_opt_validation(params_best)
validation_score 
# End time
end_time = time.time()
# Calculate total time
total_time = end_time - start_time
print("Total time taken: ", total_time)

|   iter    |  target   | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------
| 1         | 0.221     | 0.6719    | 5.905     | 1.378e+03 | 0.04113   |
| 2         | 0.221     | 0.8074    | 6.525     | 1.008e+03 | 0.05623   |
| 3         | 0.221     | 0.9896    | 2.055     | 1.339e+03 | 0.7493    |
| 4         | 0.221     | 0.3777    | 5.953     | 1.45e+03  | 0.4015    |
| 5         | 0.221     | 0.973     | 6.195     | 865.5     | 0.8152    |
| 6         | 0.221     | 0.2123    | 6.435     | 1.005e+03 | 0.818     |
| 7         | 0.221     | 0.8274    | 3.773     | 1.251e+03 | 0.1042    |
| 8         | 0.221     | 0.4118    | 2.775     | 900.8     | 0.2201    |
| 9         | 0.221     | 0.4767    | 2.621     | 964.5     | 0.01649   |
| 10        | 0.221     | 0.8978    | 6.418     | 917.3     | 0.9296    |
| 11        | 0.221     | 0.542     | 2.334     | 1.167e+03 | 0.6437    |
| 12        | 0.221     | 0.8001    | 

| 67        | 0.221     | 0.3291    | 9.642     | 800.1     | 0.5613    |
| 68        | 0.221     | 0.9899    | 9.694     | 1.499e+03 | 0.1923    |
| 69        | 0.221     | 0.03513   | 2.012     | 800.4     | 0.8605    |
| 70        | 0.221     | 0.2256    | 9.971     | 1.199e+03 | 0.5277    |
| 71        | 0.221     | 0.5392    | 9.152     | 923.4     | 0.9397    |
| 72        | 0.221     | 0.4736    | 3.583     | 1.164e+03 | 0.3491    |
| 73        | 0.221     | 0.7002    | 2.112     | 1.5e+03   | 0.184     |
| 74        | 0.221     | 0.2025    | 2.031     | 1.499e+03 | 0.324     |
| 75        | 0.221     | 0.3064    | 2.377     | 1.252e+03 | 0.4939    |
| 76        | 0.221     | 0.7084    | 7.214     | 1.121e+03 | 0.903     |
| 77        | 0.221     | 0.1086    | 5.581     | 1.486e+03 | 0.69      |
| 78        | 0.221     | 0.08402   | 8.146     | 1.409e+03 | 0.5459    |
| 79        | 0.221     | 0.2919    | 9.624     | 1.182e+03 | 0.2783    |
| 80        | 0.221     | 0.7102    | 

# XGBoost

In [12]:
import xgboost as xgb
params_best['max_depth'] = int(params_best['max_depth'])
params_best['n_estimators'] = int(params_best['n_estimators'])
params_best['subsample'] = int(params_best['subsample'])
xgb_reg= XGBRegressor(**params_best )
xgb_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.6719, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1378, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [13]:
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [14]:
y_pred=xgb_reg.predict(X_test)
y_pred1=xgb_reg.predict(X_val)
y_pred2=xgb_reg.predict(X_train)

In [15]:
print("mean_absolute_error:", mean_absolute_error(y_val, y_pred1))
print("mean_squared_error:", mean_squared_error(y_val, y_pred1))
print("rmse:", sqrt(mean_squared_error(y_val, y_pred1)))
print("r2 score:", r2_score(y_val, y_pred1))

mean_absolute_error: 2711064.2313665957
mean_squared_error: 12019826543185.9
rmse: 3466962.1490846854
r2 score: -241077285154956.9


In [16]:
print("mean_absolute_error:", mean_absolute_error(y_train, y_pred2))
print("mean_squared_error:", mean_squared_error(y_train, y_pred2))
print("rmse:", sqrt(mean_squared_error(y_train, y_pred2)))
print("r2 score:", r2_score(y_train, y_pred2))

mean_absolute_error: 2315843.4080882794
mean_squared_error: 8476037584870.367
rmse: 2911363.5267465943
r2 score: -173279902259769.4


In [17]:
print("mean_absolute_error:", mean_absolute_error(y_test, y_pred))
print("mean_squared_error:", mean_squared_error(y_test, y_pred))
print("rmse:", sqrt(mean_squared_error(y_test, y_pred)))
print("r2 score:", r2_score(y_test, y_pred))

mean_absolute_error: 2532883.462961368
mean_squared_error: 10284447504784.588
rmse: 3206937.4026919496
r2 score: -200747887692929.62
